# In this python notebook the perfomance testing is conducted

In [17]:
import os, shutil
import cv2 as cv
import numpy as np
import pywt
import plotly.graph_objects as go

from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.restoration import denoise_wavelet


In [2]:
class DenoisingMethods:
    def denoise_bilateral(self, frame):
        return cv.bilateralFilter(frame, 9, 75, 75)
    
    def denoise_non_local_means(self, frame):
        return cv.fastNlMeansDenoising(frame, None, 30, 7, 21)
    
    def denoise_wavelet(self, frame):
        return denoise_wavelet(frame, mode="soft", wavelet_levels=3, method="BayesShrink", rescale_sigma="true")

In [4]:
num_frames = 86
noisy_frames = []
for i in range(num_frames):
    frame_path = f'raw_data/raw_frames/gray_frame_{i}.jpg'
    frame = cv.imread(frame_path)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    noisy_frames.append(frame)

denoising_methods = DenoisingMethods()

denoised_frames_wavelet = [denoising_methods.denoise_wavelet(frame) for frame in noisy_frames]
denoised_frames_bilateral = [denoising_methods.denoise_bilateral(frame) for frame in noisy_frames]
denoised_frames_nl_means = [denoising_methods.denoise_non_local_means(frame) for frame in noisy_frames]



### Evaluating the perfomance of noise reduction algorithms of raw frames

#### MSE and SSIM

In [5]:

def calculate_metrics(original_frames, denoised_frames):
    mse_values = []
    ssim_values = []
    for original_frame, denoised_frame in zip(original_frames, denoised_frames):
        mse_value = mse(original_frame, denoised_frame)
        ssim_value = ssim(original_frame, denoised_frame, data_range=original_frame.max() - original_frame.min())
        mse_values.append(mse_value)
        ssim_values.append(ssim_value)
    return mse_values, ssim_values

mse_values_bilateral, ssim_values_bilateral = calculate_metrics(noisy_frames, denoised_frames_bilateral)
mse_values_nl_means, ssim_values_nl_means = calculate_metrics(noisy_frames, denoised_frames_nl_means)
mse_values_wavelet, ssim_values_wavelet = calculate_metrics(noisy_frames, denoised_frames_wavelet)

print("Bilateral Denoising:")
print("Average MSE:", sum(mse_values_bilateral) / len(mse_values_bilateral))
print("Average SSIM:", sum(ssim_values_bilateral) / len(ssim_values_bilateral))

print("Non-local means Denoising:")
print("Average MSE:", sum(mse_values_nl_means) / len(mse_values_nl_means))
print("Average SSIM:", sum(ssim_values_nl_means) / len(ssim_values_nl_means))

print("Wavelet Denoising:")
print("Average MSE:", sum(mse_values_wavelet) / len(mse_values_wavelet))
print("Average SSIM:", sum(ssim_values_wavelet) / len(ssim_values_wavelet))


fig_mse = go.Figure()

fig_mse.add_trace(go.Box(y=mse_values_bilateral, name='Bilateral', boxpoints='all'))
fig_mse.add_trace(go.Box(y=mse_values_nl_means, name='Non-Local Means', boxpoints='all'))
fig_mse.add_trace(go.Box(y=mse_values_wavelet, name='Wavelet', boxpoints='all'))

fig_mse.update_layout(title='MSE for Different Denoising Methods',
                      yaxis_title='MSE')

fig_mse.show()

fig_ssim = go.Figure()

fig_ssim.add_trace(go.Box(y=ssim_values_bilateral, name='Bilateral', boxpoints='all'))
fig_ssim.add_trace(go.Box(y=ssim_values_nl_means, name='Non-Local Means', boxpoints='all'))
fig_ssim.add_trace(go.Box(y=ssim_values_wavelet, name='Wavelet', boxpoints='all'))

fig_ssim.update_layout(title='SSIM for Different Denoising Methods',
                       yaxis_title='SSIM')

fig_ssim.show()

Bilateral Denoising:
Average MSE: 181.83546868691454
Average SSIM: 0.7510080261052201
Non-local means Denoising:
Average MSE: 207.78834319989505
Average SSIM: 0.6693191726270874
Wavelet Denoising:
Average MSE: 13778.752844986948
Average SSIM: 0.026313641922528656


#### PSNR (Peak Signal Noise Ratio)

In [18]:

def calculate_psnr(original_frames, denoised_frames):
    psnr_values = []
    for original_frame, denoised_frame in zip(original_frames, denoised_frames):
        psnr_value = psnr(original_frame, denoised_frame)
        psnr_values.append(psnr_value)
    return psnr_values

psnr_values_bilateral = calculate_psnr(noisy_frames, denoised_frames_bilateral)
psnr_values_nl_means = calculate_psnr(noisy_frames, denoised_frames_nl_means)
psnr_values_wavelet = calculate_psnr(noisy_frames, denoised_frames_wavelet)

print("Bilateral Denoising:")
print("Average PSNR:", sum(psnr_values_bilateral) / len(psnr_values_bilateral))

print("Non-local means Denoising:")
print("Average PSNR:", sum(psnr_values_nl_means) / len(psnr_values_nl_means))

print("Wavelet Denoising:")
print("Average PSNR:", sum(psnr_values_wavelet) / len(psnr_values_wavelet))


fig_mse = go.Figure()

fig_mse.add_trace(go.Box(y=psnr_values_bilateral, name='Bilateral', boxpoints='all'))
fig_mse.add_trace(go.Box(y=psnr_values_nl_means, name='Non-Local Means', boxpoints='all'))
fig_mse.add_trace(go.Box(y=psnr_values_wavelet, name='Wavelet', boxpoints='all'))

fig_mse.update_layout(title='PSNR for Different Denoising Methods',
                      yaxis_title='PSNR')

fig_mse.show()

Bilateral Denoising:
Average PSNR: 26.639702345791424
Non-local means Denoising:
Average PSNR: 25.70103895560679
Wavelet Denoising:
Average PSNR: 6.753428913852593


/var/folders/9q/1x2b1jzn4zl9khtcwg_yx7q40000gn/T/ipykernel_91394/511435656.py:4: UserWarning:

Inputs have mismatched dtype.  Setting data_range based on image_true.



#### PIQE (Perception based Image Quality Evaluator)

In [15]:
from pypiqe import piqe

piqe_values_wavelet = [piqe(frame)[0] for frame in denoised_frames_wavelet]
piqe_values_bilateral = [piqe(frame)[0] for frame in denoised_frames_bilateral]
piqe_values_nl_means = [piqe(frame)[0] for frame in denoised_frames_nl_means]

print("Bilateral Denoising:")
print("Average PIQE:", sum(piqe_values_bilateral) / len(piqe_values_bilateral))

print("Non-local means Denoising:")
print("Average PIQE:", sum(piqe_values_nl_means) / len(piqe_values_nl_means))

print("Wavelet Denoising:")
print("Average PIQE:", sum(piqe_values_wavelet) / len(piqe_values_wavelet))

fig_piqe = go.Figure()

fig_piqe.add_trace(go.Box(y=piqe_values_bilateral, name='Bilateral', boxpoints='all'))
fig_piqe.add_trace(go.Box(y=piqe_values_nl_means, name='Non-Local Means', boxpoints='all'))
fig_piqe.add_trace(go.Box(y=piqe_values_wavelet, name='Wavelet', boxpoints='all'))

fig_piqe.update_layout(title='PIQE for Different Denoising Methods',
                       yaxis_title='PIQE score')

fig_piqe.show()

Bilateral Denoising:
Average PIQE: 70.91700543018948
Non-local means Denoising:
Average PIQE: 78.10480438318214
Wavelet Denoising:
Average PIQE: 34.09018232725886


### Evaluating the impact of sharpening process on the perfomance of Genetic Algorithm which is implemented in IIR filter